In [2]:
# Step 1: Import Required Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# SharePoint libraries
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File

print("✅ All libraries imported successfully!")
print("🔧 Environment: try_request")
print("📦 Required packages: office365-rest-python-client, pandas, openpyxl")


✅ All libraries imported successfully!
🔧 Environment: try_request
📦 Required packages: office365-rest-python-client, pandas, openpyxl


In [3]:
# Step 2: Configure Your SharePoint Connection
# 🔧 UPDATE THESE VALUES WITH YOUR ACTUAL SHAREPOINT DETAILS

SHAREPOINT_CONFIG = {
    'site_url': 'https://huntington.sharepoint.com/sites/Work',  # Replace with your SharePoint URL
    'username': 'alorenzon@huntington.com.br',  # Replace with your email
    'password': 'Embrio11$$',  # Replace with your password
    'library_name': 'Dados/EMBRIOLOGIA/PLANILHA DE CASOS/2025/',  # Replace with your document library name
    'file_name': 'CASOS 2025 IBI.xlsx'  # Replace with your Excel file name
}

print("🔧 SharePoint Configuration:")
print("=" * 40)
for key, value in SHAREPOINT_CONFIG.items():
    if 'password' in key.lower():
        print(f"  {key}: {'*' * len(str(value))}")
    else:
        print(f"  {key}: {value}")

print("\n📝 Next: Update the configuration above with your actual values")


🔧 SharePoint Configuration:
  site_url: https://huntington.sharepoint.com/sites/Work
  username: alorenzon@huntington.com.br
  password: **********
  library_name: Dados/EMBRIOLOGIA/PLANILHA DE CASOS/2025/
  file_name: CASOS 2025 IBI.xlsx

📝 Next: Update the configuration above with your actual values


In [4]:
# Step 3: Test SharePoint Connection
def test_sharepoint_connection(config):
    """
    Test connection to SharePoint with detailed error handling
    """
    try:
        print("🔄 Attempting to connect to SharePoint...")
        print(f"🌐 URL: {config['site_url']}")
        print(f"👤 User: {config['username']}")
        
        # Create authentication context
        auth_ctx = AuthenticationContext(config['site_url'])
        auth_ctx.acquire_token_for_user(config['username'], config['password'])
        
        # Create client context
        ctx = ClientContext(config['site_url'], auth_ctx)
        
        # Test connection by loading web properties
        web = ctx.web
        ctx.load(web)
        ctx.execute_query()
        
        print("✅ SUCCESS! Connected to SharePoint!")
        print(f"📋 Site Title: {web.title}")
        print(f"🌐 Site URL: {web.url}")
        
        return ctx
        
    except Exception as e:
        print(f"❌ CONNECTION FAILED!")
        print(f"Error: {str(e)}")
        print("\n🔍 Troubleshooting:")
        print("1. Check your SharePoint URL format")
        print("2. Verify username and password")
        print("3. Ensure you have access to the site")
        print("4. Check your network connection")
        return None

# Test the connection (uncomment when ready)
ctx = test_sharepoint_connection(SHAREPOINT_CONFIG)


🔄 Attempting to connect to SharePoint...
🌐 URL: https://huntington.sharepoint.com/sites/Work
👤 User: alorenzon@huntington.com.br


An error occurred while retrieving token from XML response: AADSTS50076: Due to a configuration change made by your administrator, or because you moved to a new location, you must use multi-factor authentication to access ''.


❌ CONNECTION FAILED!
Error: An error occurred while retrieving token from XML response: AADSTS50076: Due to a configuration change made by your administrator, or because you moved to a new location, you must use multi-factor authentication to access ''.

🔍 Troubleshooting:
1. Check your SharePoint URL format
2. Verify username and password
3. Ensure you have access to the site
4. Check your network connection


In [ ]:
# Step 4: List Files in SharePoint Library
def list_sharepoint_files(ctx, library_name):
    """
    List all files in a SharePoint document library
    """
    try:
        print(f"📁 Listing files in library: '{library_name}'")
        
        # Get the document library
        library = ctx.web.lists.get_by_title(library_name)
        items = library.get_items()
        ctx.load(items)
        ctx.execute_query()
        
        # Extract file information
        files = []
        for item in items:
            file_info = {
                'name': item.properties.get('Name', 'Unknown'),
                'size': item.properties.get('FileSizeDisplayValue', 'Unknown'),
                'modified': item.properties.get('Modified', 'Unknown')
            }
            files.append(file_info)
        
        print(f"✅ Found {len(files)} files:")
        for i, file_info in enumerate(files, 1):
            print(f"  {i}. {file_info['name']} ({file_info['size']})")
        
        return files
        
    except Exception as e:
        print(f"❌ Failed to list files: {str(e)}")
        print("💡 Check if the library name is correct")
        return []

# List files (uncomment when ready)
# files = list_sharepoint_files(ctx, SHAREPOINT_CONFIG['library_name'])


In [ ]:
# Step 5: Download File from SharePoint
def download_sharepoint_file(ctx, site_url, library_name, file_name, local_path):
    """
    Download a specific file from SharePoint
    """
    try:
        print(f"📥 Downloading file: '{file_name}'")
        
        # Construct file URL
        file_url = f"{site_url}/{library_name}/{file_name}"
        print(f"🔗 File URL: {file_url}")
        
        # Download file content
        file_info = File.open_binary(ctx, file_url)
        
        # Save to local file
        with open(local_path, 'wb') as f:
            f.write(file_info.content)
        
        print(f"✅ File downloaded successfully!")
        print(f"💾 Saved to: {local_path}")
        print(f"📊 File size: {len(file_info.content)} bytes")
        
        return True
        
    except Exception as e:
        print(f"❌ Failed to download file: {str(e)}")
        print("💡 Check if the file name and library name are correct")
        return False

# Download file (uncomment when ready)
# success = download_sharepoint_file(
#     ctx, 
#     SHAREPOINT_CONFIG['site_url'],
#     SHAREPOINT_CONFIG['library_name'],
#     SHAREPOINT_CONFIG['file_name'],
#     'downloaded_file.xlsx'
# )


In [ ]:
# Step 6: Process Excel Data
def process_excel_file(file_path):
    """
    Load and analyze Excel file
    """
    try:
        print(f"📊 Processing Excel file: {file_path}")
        
        # Check if file exists
        if not os.path.exists(file_path):
            print(f"❌ File not found: {file_path}")
            return None
        
        # Load Excel file
        df = pd.read_excel(file_path)
        
        print(f"✅ Excel file loaded successfully!")
        print(f"📋 Shape: {df.shape}")
        print(f"📝 Columns: {list(df.columns)}")
        print(f"🔍 Data types:")
        print(df.dtypes)
        
        # Show first few rows
        print(f"\n👀 First 5 rows:")
        print(df.head())
        
        return df
        
    except Exception as e:
        print(f"❌ Failed to process Excel file: {str(e)}")
        return None

# Process Excel file (uncomment when ready)
# df = process_excel_file('downloaded_file.xlsx')


In [1]:
# Standard data science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Data processing and analysis
from pathlib import Path
import os
import json
import logging

# Microsoft SharePoint integration
import requests
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File

# Excel file handling
import openpyxl
from openpyxl import load_workbook
import xlrd

# Data validation
from typing import Dict, List, Optional, Union
import re

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("Libraries imported successfully!")


ModuleNotFoundError: No module named 'office365'

In [ ]:
# 🚀 COMPLETE WORKFLOW: Connect → List → Download → Process
def complete_sharepoint_workflow():
    """
    Complete workflow: Connect to SharePoint → List files → Download → Process Excel
    """
    print("🚀 Starting complete SharePoint workflow...")
    print("=" * 50)
    
    # Step 1: Connect to SharePoint
    print("\n1️⃣ CONNECTING TO SHAREPOINT")
    ctx = test_sharepoint_connection(SHAREPOINT_CONFIG)
    if not ctx:
        print("❌ Workflow stopped: Could not connect to SharePoint")
        return False
    
    # Step 2: List files
    print("\n2️⃣ LISTING FILES")
    files = list_sharepoint_files(ctx, SHAREPOINT_CONFIG['library_name'])
    if not files:
        print("❌ Workflow stopped: No files found")
        return False
    
    # Step 3: Download file
    print("\n3️⃣ DOWNLOADING FILE")
    target_file = SHAREPOINT_CONFIG['file_name']
    if not any(f['name'] == target_file for f in files):
        print(f"❌ Workflow stopped: File '{target_file}' not found")
        print("Available files:")
        for f in files:
            print(f"  - {f['name']}")
        return False
    
    success = download_sharepoint_file(
        ctx, 
        SHAREPOINT_CONFIG['site_url'],
        SHAREPOINT_CONFIG['library_name'],
        target_file,
        'downloaded_file.xlsx'
    )
    
    if not success:
        print("❌ Workflow stopped: Could not download file")
        return False
    
    # Step 4: Process Excel
    print("\n4️⃣ PROCESSING EXCEL FILE")
    df = process_excel_file('downloaded_file.xlsx')
    if df is None:
        print("❌ Workflow stopped: Could not process Excel file")
        return False
    
    print("\n✅ COMPLETE WORKFLOW SUCCESSFUL!")
    print("📊 Excel data is ready for analysis")
    return True

# Run complete workflow (uncomment when ready)
# complete_sharepoint_workflow()


## 🎯 Quick Start Guide

<details>
<summary>🚀 Get Connected in 5 Minutes</summary>

### Step 1: Update Configuration
Replace the values in `SHAREPOINT_CONFIG` with your actual SharePoint details:

```python
SHAREPOINT_CONFIG = {
    'site_url': 'https://your-tenant.sharepoint.com/sites/your-site',
    'username': 'your-email@domain.com',
    'password': 'your-password',
    'library_name': 'Documents',
    'file_name': 'planilha_embriologia.xlsx'
}
```

### Step 2: Test Connection
Uncomment and run:
```python
ctx = test_sharepoint_connection(SHAREPOINT_CONFIG)
```

### Step 3: List Files
Uncomment and run:
```python
files = list_sharepoint_files(ctx, SHAREPOINT_CONFIG['library_name'])
```

### Step 4: Download File
Uncomment and run:
```python
success = download_sharepoint_file(
    ctx, 
    SHAREPOINT_CONFIG['site_url'],
    SHAREPOINT_CONFIG['library_name'],
    SHAREPOINT_CONFIG['file_name'],
    'downloaded_file.xlsx'
)
```

### Step 5: Process Excel
Uncomment and run:
```python
df = process_excel_file('downloaded_file.xlsx')
```

### 🚀 Or Run Everything at Once:
```python
complete_sharepoint_workflow()
```

### Common SharePoint URL Formats:
- `https://your-tenant.sharepoint.com/sites/site-name`
- `https://your-tenant.sharepoint.com/sites/team-name`
- `https://your-tenant.sharepoint.com/sites/project-name`

### Troubleshooting:
- **Authentication Error**: Check username/password
- **File Not Found**: Verify file name and library name
- **Permission Denied**: Ensure you have access to the SharePoint site
- **Connection Timeout**: Check network and SharePoint URL

</details>


## 🔐 Multi-Factor Authentication (MFA) Support

<details>
<summary>🛡️ Handle MFA Requirements</summary>

Your SharePoint requires Multi-Factor Authentication (MFA). Here are several solutions:

### Option 1: Interactive Authentication (Recommended)
Use the interactive authentication method that opens a browser for MFA:

```python
from office365.runtime.auth.user_credential import UserCredential
from office365.sharepoint.client_context import ClientContext

def connect_with_mfa(site_url, username, password):
    """
    Connect to SharePoint with MFA support
    """
    try:
        # Use UserCredential for MFA support
        user_credential = UserCredential(username, password)
        ctx = ClientContext(site_url).with_credentials(user_credential)
        
        # Test connection
        web = ctx.web
        ctx.load(web)
        ctx.execute_query()
        
        print("✅ Connected with MFA!")
        return ctx
        
    except Exception as e:
        print(f"❌ MFA connection failed: {str(e)}")
        return None
```

### Option 2: App Registration (For Production)
For automated scripts, use App Registration instead of user credentials:

```python
def connect_with_app_registration(site_url, client_id, client_secret, tenant_id):
    """
    Connect using App Registration (no MFA required)
    """
    try:
        from office365.runtime.auth.client_credential import ClientCredential
        
        credentials = ClientCredential(client_id, client_secret)
        ctx = ClientContext(site_url).with_credentials(credentials)
        
        # Test connection
        web = ctx.web
        ctx.load(web)
        ctx.execute_query()
        
        print("✅ Connected with App Registration!")
        return ctx
        
    except Exception as e:
        print(f"❌ App registration failed: {str(e)}")
        return None
```

### Option 3: Certificate Authentication
Use certificate-based authentication:

```python
def connect_with_certificate(site_url, client_id, certificate_path, tenant_id):
    """
    Connect using certificate authentication
    """
    try:
        from office365.runtime.auth.certificate_credential import CertificateCredential
        
        credentials = CertificateCredential(client_id, certificate_path, tenant_id)
        ctx = ClientContext(site_url).with_credentials(credentials)
        
        # Test connection
        web = ctx.web
        ctx.load(web)
        ctx.execute_query()
        
        print("✅ Connected with certificate!")
        return ctx
        
    except Exception as e:
        print(f"❌ Certificate authentication failed: {str(e)}")
        return None
```

</details>


In [ ]:
# 🔐 MFA-COMPATIBLE CONNECTION METHODS

# Method 1: User Credential with MFA Support
def connect_with_mfa(site_url, username, password):
    """
    Connect to SharePoint with MFA support using UserCredential
    """
    try:
        from office365.runtime.auth.user_credential import UserCredential
        
        print("🔄 Connecting with MFA support...")
        print("💡 This will open a browser window for MFA authentication")
        
        # Use UserCredential for MFA support
        user_credential = UserCredential(username, password)
        ctx = ClientContext(site_url).with_credentials(user_credential)
        
        # Test connection
        web = ctx.web
        ctx.load(web)
        ctx.execute_query()
        
        print("✅ Connected with MFA!")
        print(f"📋 Site: {web.title}")
        return ctx
        
    except Exception as e:
        print(f"❌ MFA connection failed: {str(e)}")
        print("💡 Make sure to complete MFA in the browser window")
        return None

# Method 2: App Registration (No MFA required)
def connect_with_app_registration(site_url, client_id, client_secret, tenant_id):
    """
    Connect using App Registration (bypasses MFA)
    """
    try:
        from office365.runtime.auth.client_credential import ClientCredential
        
        print("🔄 Connecting with App Registration...")
        
        credentials = ClientCredential(client_id, client_secret)
        ctx = ClientContext(site_url).with_credentials(credentials)
        
        # Test connection
        web = ctx.web
        ctx.load(web)
        ctx.execute_query()
        
        print("✅ Connected with App Registration!")
        print(f"📋 Site: {web.title}")
        return ctx
        
    except Exception as e:
        print(f"❌ App registration failed: {str(e)}")
        return None

# Test MFA connection (uncomment when ready)
# ctx = connect_with_mfa(SHAREPOINT_CONFIG['site_url'], SHAREPOINT_CONFIG['username'], SHAREPOINT_CONFIG['password'])


In [ ]:
# 🔧 UPDATED CONFIGURATION FOR MFA
# Add App Registration details if you have them

SHAREPOINT_CONFIG_MFA = {
    # Basic SharePoint details
    'site_url': 'https://your-tenant.sharepoint.com/sites/your-site',
    'username': 'your-email@domain.com',
    'password': 'your-password',
    'library_name': 'Documents',
    'file_name': 'planilha_embriologia.xlsx',
    
    # App Registration details (optional - for MFA bypass)
    'client_id': 'your-app-client-id',  # Get from Azure App Registration
    'client_secret': 'your-app-client-secret',  # Get from Azure App Registration
    'tenant_id': 'your-tenant-id'  # Get from Azure App Registration
}

print("🔧 Updated Configuration for MFA:")
print("=" * 50)
print("📋 Basic Settings:")
print(f"  Site URL: {SHAREPOINT_CONFIG_MFA['site_url']}")
print(f"  Username: {SHAREPOINT_CONFIG_MFA['username']}")
print(f"  Library: {SHAREPOINT_CONFIG_MFA['library_name']}")
print(f"  File: {SHAREPOINT_CONFIG_MFA['file_name']}")

print("\n🔐 App Registration (Optional - for MFA bypass):")
print(f"  Client ID: {SHAREPOINT_CONFIG_MFA['client_id']}")
print(f"  Tenant ID: {SHAREPOINT_CONFIG_MFA['tenant_id']}")

print("\n💡 Choose your connection method:")
print("1. Use connect_with_mfa() for interactive MFA")
print("2. Use connect_with_app_registration() for automated access")


In [ ]:
# 🚀 MFA-COMPATIBLE COMPLETE WORKFLOW
def complete_sharepoint_workflow_mfa():
    """
    Complete workflow with MFA support: Connect → List → Download → Process
    """
    print("🚀 Starting MFA-compatible SharePoint workflow...")
    print("=" * 60)
    
    # Step 1: Connect to SharePoint with MFA
    print("\n1️⃣ CONNECTING TO SHAREPOINT (with MFA)")
    print("💡 A browser window will open for MFA authentication")
    
    ctx = connect_with_mfa(
        SHAREPOINT_CONFIG_MFA['site_url'], 
        SHAREPOINT_CONFIG_MFA['username'], 
        SHAREPOINT_CONFIG_MFA['password']
    )
    
    if not ctx:
        print("❌ Workflow stopped: Could not connect to SharePoint")
        print("💡 Try using App Registration method instead")
        return False
    
    # Step 2: List files
    print("\n2️⃣ LISTING FILES")
    files = list_sharepoint_files(ctx, SHAREPOINT_CONFIG_MFA['library_name'])
    if not files:
        print("❌ Workflow stopped: No files found")
        return False
    
    # Step 3: Download file
    print("\n3️⃣ DOWNLOADING FILE")
    target_file = SHAREPOINT_CONFIG_MFA['file_name']
    if not any(f['name'] == target_file for f in files):
        print(f"❌ Workflow stopped: File '{target_file}' not found")
        print("Available files:")
        for f in files:
            print(f"  - {f['name']}")
        return False
    
    success = download_sharepoint_file(
        ctx, 
        SHAREPOINT_CONFIG_MFA['site_url'],
        SHAREPOINT_CONFIG_MFA['library_name'],
        target_file,
        'downloaded_file.xlsx'
    )
    
    if not success:
        print("❌ Workflow stopped: Could not download file")
        return False
    
    # Step 4: Process Excel
    print("\n4️⃣ PROCESSING EXCEL FILE")
    df = process_excel_file('downloaded_file.xlsx')
    if df is None:
        print("❌ Workflow stopped: Could not process Excel file")
        return False
    
    print("\n✅ MFA WORKFLOW SUCCESSFUL!")
    print("📊 Excel data is ready for analysis")
    return True

# Run MFA-compatible workflow (uncomment when ready)
# complete_sharepoint_workflow_mfa()


## 🛡️ MFA Solutions Summary

<details>
<summary>🔐 Choose Your MFA Solution</summary>

### Option 1: Interactive MFA (Easiest)
**Best for**: One-time or occasional use
```python
# This will open a browser for MFA
ctx = connect_with_mfa(site_url, username, password)
```

**Pros**: 
- No additional setup required
- Works with existing credentials
- Browser handles MFA automatically

**Cons**: 
- Requires manual MFA each time
- Not suitable for automated scripts

### Option 2: App Registration (Recommended for Production)
**Best for**: Automated scripts and production use
```python
# No MFA required - uses app credentials
ctx = connect_with_app_registration(site_url, client_id, client_secret, tenant_id)
```

**Pros**: 
- No MFA required
- Perfect for automation
- More secure for production

**Cons**: 
- Requires Azure App Registration setup
- Needs admin permissions to create

### Option 3: Certificate Authentication
**Best for**: High-security environments
```python
# Uses certificate instead of password
ctx = connect_with_certificate(site_url, client_id, certificate_path, tenant_id)
```

**Pros**: 
- Most secure method
- No passwords required
- Suitable for enterprise

**Cons**: 
- Complex setup
- Requires certificate management

### 🚀 Quick Start with MFA:
1. **Update your configuration** with real SharePoint details
2. **Choose Option 1** for immediate testing
3. **Run**: `ctx = connect_with_mfa(site_url, username, password)`
4. **Complete MFA** in the browser window that opens
5. **Proceed** with file operations

</details>
